# Keras Tuner

- A package to tune Keras hyperparameters

In [0]:
%tensorflow_version 2.x

In [0]:
!pip install keras-tuner

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist


Label   Description
- 0   T-shirt/top
- 1   Trouser
- 2   Pullover
- 3   Dress
- 4   Coat
- 5   Sandal
- 6   Shirt
- 7   Sneaker
- 8   Bag
- 9   Ankle boot


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
import time
import pickle

In [0]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# Loading Data

In [0]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [166]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [0]:
x_train = tf.expand_dims(x_train, -1)
x_test = tf.expand_dims(x_test, -1)

In [168]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [0]:
plt.imshow(x_train[1], cmap="gray")
plt.show()

# Non-keras Tuner Model

In [0]:
def build_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape = x_train.shape[1:]))
    model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
    
    model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu',))
    model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

    return model

In [0]:
model = build_model()

In [0]:
model.summary()

In [0]:
history = model.fit(x_train, y_train, batch_size=32, epochs=1, verbose=1, validation_split=0.2, callbacks=None)

# With Keras-Tuner Model

In [0]:
def build_model_tuned(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(hp.Int("input_units", min_value=32, max_value=256, step=32), (3,3), input_shape=x_train.shape[1:]))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
    
    for i in range(hp.Int('n_layers',1,4)):
        model.add(tf.keras.layers.Conv2D(hp.Int(f'conv_{i}_units', min_value=32, max_value=256, step=32), (3,3)))
        model.add(tf.keras.layers.Activation('relu'))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

    return model

In [0]:
LOG_DIR = f"{int(time.time())}"

# Metrics and Performance

In [0]:
tuner = RandomSearch(build_model_tuned, objective="val_acc", 
                     max_trials = 1,
                     executions_per_trial = 1,
                     directory = LOG_DIR)

In [0]:
tuner.search(x=x_train, y=y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

In [0]:
tuner.search_space_summary()

In [0]:
tuner.results_summary()

In [0]:
print(tuner.get_best_hyperparameters()[0].values)

{'input_units': 192, 'n_layers': 2, 'conv_0_units': 160, 'conv_1_units': 32}


In [0]:
best_model = tuner.get_best_models()[0]

In [0]:
best_model.summary()

# In Short what we did

In [0]:
optimizer = hp.Choice('optimizer', ['adam', 'sgd'])

In [0]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = tf.expand_dims(x_train, -1)
x_test = tf.expand_dims(x_test, -1)

In [0]:
def build_tuned_model2(hp, num_classes=10):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(hp.Int('input_units', min_value=32, max_value=256, step=32), (3,3), input_shape = x_train.shape[1:]))
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    
    for i in range(hp.Int('n_layers',1, 4)):
        model.add(tf.keras.layers.Conv2D(hp.Int(f'conv_{i}_units', min_value=32, max_value=256, step=32), (3,3)))
        model.add(tf.keras.layers.Activation('elu'))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='elu'))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])

    return model

In [0]:
tuner = RandomSearch(build_tuned_model2, objective='val_acc', max_trials=20, executions_per_trial=2, directory=LOG_DIR)

INFO:tensorflow:Reloading Oracle from 1577186939/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from 1577186939/untitled_project/tuner0.json


In [0]:
tuner.search_space_summary()

In [0]:
tuner.search(x=x_train, y=y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))

# Loading and Saving the Model using Pickle

In [0]:
with open(f"tuner_{int(time.time())}.pkl", "wb") as f:
    pickle.dump(tuner,f)

In [114]:
tuner = pickle.load(open("/content/tuner_1577190691.pkl","rb"))
tuner.get_best_hyperparameters()[0].values

{'conv_0_units': 128,
 'conv_1_units': 96,
 'conv_2_units': 256,
 'conv_3_units': 96,
 'input_units': 64,
 'n_layers': 1}

In [0]:
best_model2 = tuner.get_best_models()[0]

In [116]:
best_model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
activation_1 (Activation)    (None, 11, 11, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

# Some extra Parameters to tune

- Use hp.Choice to provide choice to the hyperparametrs

In [0]:
def build_model_add(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=hp.Int('input_units', min_value=32, max_value=512, step=32),kernel_size=(3,3),
                                     activation='relu', input_shape=x_train.shape[1:]))
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    # To customize the learning rate

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-4, 1e-5, 1e-6])), 
                  loss="sparse_categorical_crossentropy", metrics=['acc'])
    
    return model


In [0]:
tuned_model3 = RandomSearch(
    build_model_add,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

# SubClassing to build a new HyperTuner Model

In [0]:
from kerastuner import HyperModel

In [0]:
class MyHyperModel(HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes
    
    def build(self, hp):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='elu'))
        model.add(tf.keras.layers.Dense(self.num_classes, activation='softmax'))

        model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-4,1e-5,1e-6])),
                      loss='categorical_crossentropy', metrics=['acc'])
        
        return model


In [0]:
hypermodel = MyHyperModel(num_classes=10)

In [141]:
tuner = RandomSearch(hypermodel, objective='val_acc', max_trials=10, directory='LOG_DIR', project_name='new_project1')

INFO:tensorflow:Reloading Oracle from LOG_DIR/new_project1/oracle.json


In [0]:
tuner.search(x=x_train, y=y_train, epochs=10, validation_data=(x_test, y_test))

# Using Premade Tunable Applications Hyper-Resnet and HyperXception

- These are ready-to-use hypermodels for computer vision.

- They come pre-compiled with loss="categorical_crossentropy" and metrics=["accuracy"].

In [0]:
from kerastuner.applications import HyperResNet
from kerastuner import HyperParameters
from kerastuner.tuners import Hyperband

In [0]:
hypermodel = HyperResNet(input_shape=(28,28,1), classes=10)

tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=40,
    directory='my_dir',
    project_name='helloworld')


In [0]:
tuner.search(x_train, y_train,
             epochs=20,
             validation_data=(x_test, y_test))

# Restricting the Search space

In [0]:
hypermodel = HyperXception(input_shape=(28, 28, 1), classes=10)
hp = HyperParameters()

In [173]:
hp.Choice('learning_rate', values=[1e-4,1e-5,1e-6])

0.0001

In [0]:
tuner = Hyperband(hypermodel, hyperparameters=hp, tune_new_entries=False, objective='val_acc', max_epochs=40, directory='my_dir',
                  project_name='try3')

In [0]:
tuner.search(x_train, y_train, epochs=20,  validation_data=(x_test, y_test))

# Over-riding the Compilation Arguments

In [0]:
hypermodel = HyperXception(input_shape=(28, 28, 1), classes=10)

In [181]:
tuner = Hyperband(hypermodel, optimizer=tf.keras.optimizers.Adam(1e-3), loss='mse',
                  metrics=[tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')], objective='val_acc',
                  max_epochs=40, directory='my_dr', project_name="hello")

INFO:tensorflow:Reloading Oracle from my_dr/hello/oracle.json


In [0]:
tuner.search(x_train, y_train, epochs=20, validation_data=(x_test,y_test))